In [ ]:
import numpy as np

from utils import gaussjordan

from LinearCode import LinearCode


class QC_LDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p
        
    """

    def __init__(self, G, H):
        super().__init__(G, H)
        
    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"
        
        n0 = n // p
        assert w > 2*n0, "not enough code weight"
        
        fine = False
        
        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None
            
            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)
            
            for i in range(n0):
                circ = vector[i*p:(i+1)*p]
                
                if sum(circ) < 2:
                    inverse_block = None
                    break
        
                block = QC_LDPC._get_circulant_block(circ)
                blocks.append(block)
                    
                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)
                
                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P
                
            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False
        
        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T            

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)
        
        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"
        
        return cls(G, H)
    
    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)
        
        for i in range(N):
            block[i] = np.roll(polynom, i)
            
        return block



# EXAMPLE USAGE:

n = 16
p = 8
w = 7

qc_ldpc = QC_LDPC.from_params(n, p, w)
print(qc_ldpc.getG())
print(qc_ldpc.getG().shape)

In [ ]:
print(qc_ldpc.getH())
print(qc_ldpc.getH().shape)

In [208]:
word = np.random.randint(2, size=qc_ldpc.getG().shape[0])

In [ ]:
word

In [ ]:
print(len(word))

In [ ]:
encoded = qc_ldpc.encode(word)
print(encoded)
print(len(encoded))

In [ ]:
# error vector size n with t or less errors
errors_num = 2
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e)
np.random.shuffle(e)
print(e)
print(len(e))

In [ ]:
corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

In [ ]:
decoded = qc_ldpc.decode(np.copy(corrupted))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

In [ ]:
word

In [ ]:
try:
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

In [ ]:
from google.colab import drive
import os
import shutil
drive.mount('/content/gdrive')

In [ ]:
os.chdir('/content/gdrive/My Drive/Animal Species')

In [ ]:
ls

In [144]:
value = 0 
dico = {}
for filename in os.listdir('/content/gdrive/My Drive/Animal Species'):
        if(os.path.isdir(filename)):
        dico[value]  = filename
        value = value + 1

In [ ]:
 for i in dico.items():
    print(i)    

In [ ]:
y = corrupted
print(y)
len(y)

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
lis

In [150]:
import hashlib
import os


def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

In [ ]:
list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

In [ ]:
y

In [ ]:
liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

In [ ]:
len(liste)

In [154]:
i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = 'C:/Users/Willy/Desktop/github/Animal Species/' + dico[i]
        os.chdir(my_path)
        #print(my_path + file)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

In [158]:
msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

In [ ]:
msg_secret

In [160]:
len(msg_secret)

226

In [ ]:
np.array(msg_secret, dtype = 'i')

In [ ]:
decoded = qc_ldpc.decode(np.copy(np.array(msg_secret, dtype = 'i')))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

In [ ]:
word

In [164]:
try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

# #Code script for time evaluation

In [ ]:
import time

start = time.time()

import numpy as np

from utils import gaussjordan

from LinearCode import LinearCode


class QC_LDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p
        
    """

    def __init__(self, G, H):
        super().__init__(G, H)
        
    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"
        
        n0 = n // p
        assert w > 2*n0, "not enough code weight"
        
        fine = False
        
        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None
            
            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)
            
            for i in range(n0):
                circ = vector[i*p:(i+1)*p]
                
                if sum(circ) < 2:
                    inverse_block = None
                    break
        
                block = QC_LDPC._get_circulant_block(circ)
                blocks.append(block)
                    
                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)
                
                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P
                
            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False
        
        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T            

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)
        
        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"
        
        return cls(G, H)
    
    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)
        
        for i in range(N):
            block[i] = np.roll(polynom, i)
            
        return block



# EXAMPLE USAGE:

n = 16
p = 8
w = 7

qc_ldpc = QC_LDPC.from_params(n, p, w)
print(qc_ldpc.getG())
print(qc_ldpc.getG().shape)

print(qc_ldpc.getH())
print(qc_ldpc.getH().shape)

word = np.random.randint(2, size=qc_ldpc.getG().shape[0])



word

print(len(word))

encoded = qc_ldpc.encode(word)
print(encoded)
print(len(encoded))

# error vector size n with t or less errors
errors_num = 2
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e)
np.random.shuffle(e)
print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

import os
import shutil

os.chdir('/content/gdrive/My Drive/Animal Species')

#print(ls)

value = 0 
dico = {}
for filename in os.listdir('/content/gdrive/My Drive/Animal Species'):
        if(os.path.isdir(filename)):
            dico[value]  = filename
            value = value + 1
        
        
for i in dico.items():
    print(i)

y = corrupted
print(y)
len(y)

i = 0
while i < len(y):
    my_path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    os.chdir(my_path)
    if os.path.isfile(my_path + '/file.txt'):
        os.remove(my_path + '/file.txt')
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

print(lis)

import hashlib
import os


def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

print(y)

liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = 'C:/Users/Willy/Desktop/github/Animal Species/' + dico[i]
        os.chdir(my_path)
        #print(my_path + file)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/My Drive/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1



list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

print(np.array(msg_secret, dtype = 'i'))


decoded = qc_ldpc.decode(np.copy(np.array(msg_secret, dtype = 'i')))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

print(word)

try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")



end = time.time()
print('time = ',end - start)